In [1]:
import astroprov
from astropy.io import fits
from astropy.table import Table
from astropy.io import ascii
from astropy.stats import sigma_clipped_stats
from astropy.coordinates import SkyCoord

import matplotlib.pyplot as plt
import numpy as np
import os
from regions.core import PixCoord
from regions.shapes.circle import CirclePixelRegion
import subprocess
import time
import matplotlib.text as mpl_text
import math

In [2]:
imageRefernce = '/home/mj1e16/moleGazer/Photos/F/29018.jpg'
imageAlign =  '/home/mj1e16/moleGazer/Photos/F/79948.jpg'

imageRefernceFits = '/home/mj1e16/moleGazer/Photos/F/fitsImages/16bit/29018InvertedAverageCropped16.fits'
imageAlignFits =  '/home/mj1e16/moleGazer/Photos/F/fitsImages/16bit/79948InvertedAverageCropped16.fits'

## max setting [6.444444444444445, 21.842105263157894, 'gauss_5.0_9x9.conv']

refTab = Table.read('/home/mj1e16/moleGazer/Photos/F/fitsImages/simMolePos/29018InvertedAverageCropped166.44444444444_21.8421052632_gauss_5.0_9x9.conv.cat',format='ascii.sextractor')
alignTab = Table.read('/home/mj1e16/moleGazer/Photos/F/fitsImages/simMolePos/79948InvertedAverageCropped166.44444444444_21.8421052632_gauss_5.0_9x9.conv.cat',format='ascii.sextractor')

In [3]:
def makeDS9RegFile(sexTabList,fileNameBase,tabType,colour='red',radius='10'):
    for tables in range(len(sexTabList)):
        if tabType == 'dao':
            xcoords = sexTabList[tables]['xcentroid']
            ycoords = sexTabList[tables]['ycentroid']
        elif tabType == 'sexMedian':
            ra = sexTabList[tables]['RA']
            dec = sexTabList[tables]['DEC']
        elif tabType == 'sex':
            xcoords = sexTabList[tables]['X_IMAGE']
            ycoords = sexTabList[tables]['Y_IMAGE']
        elif tabType == 'iraf':
            xcoords = sexTabList[tables]['X_POS']
            ycoords = sexTabList[tables]['Y_POS']        
        else:
            print('tab type error')
            break
        bigString = 'image\n'
        for x in range(len(xcoords)):
            bigString += 'circle({},{},'.format(xcoords[x],ycoords[x])+radius+') # color={}\n'.format(colour)
        fileName = fileNameBase + '.reg'
        with open(fileName,'w') as f:
            f.write(bigString)
            
    return fileName

In [ ]:
makeDS9RegFile([alignTab],'/home/mj1e16/moleGazer/MoleGazer/79948Regions','sex')
makeDS9RegFile([refTab],'/home/mj1e16/moleGazer/MoleGazer/29018Regions','sex')

In [4]:
# could use a re-work

def findTestSources(sources,lowx, highx, lowy, highy, imageNo):
    """
    Searches for bright objects to be used for pattern recognition within a specified region of the reference image.

    :param sources: Table of object data
    :param lowx: Lower X limit of for searching region region
    :param highx: Upper X limit
    :param lowy: Lower Y limit
    :param highy: Upper Y limit
    :param imageNo: Reference image number
    :return: Indexes in sources of (up to) the brightest 15 objects contained within the specified region
    """
    testsources = {}
    mag = []
    number = []
    #for x in range(sources[imageNo][-1][0]):
    for x in range(len(sources[imageNo])):
        x1 = sources[imageNo]['X_IMAGE'][x]
        y1 = sources[imageNo]['Y_IMAGE'][x]
        if x1 > lowx and x1 < highx and y1 > lowy and y1 < highy:
            testsources[x] = sources[imageNo][x]
            mag.append(sources[imageNo]['MAG_BEST'][x])
            number.append(x)
    Z2 = [x for _,x in sorted(zip(mag,number))]
    samplestars = Z2[0:15]
    
    #astroprov.provcall([sources,lowx, highx, lowy, highy, imageNo],[samplestars],"findTestSources_PythonCode2PythonCode_SQ_tmpl.provn","findTestSources")
    return samplestars

In [5]:
def getImageData(imageName,extension):
    hdu = fits.open(imageName)
    imageData = hdu[extension].data
    return imageData

In [6]:
def ds9File2Lists(fileName):
    with open(fileName,'r') as f:
        data = f.readlines()

    ycoord = []
    xcoord = []
    radius = []
    starLocations = []
    for x in data[:-1]:
        try:
            locator1 = '('
            locator2 = ','
            locator3 = ')'
            loc1 = x.index(locator1) +1
            #print(x)
            loc2 = x[loc1:].index(locator2) + loc1 +1
            loc3 = x[loc2:].index(locator2) + loc2
            loc4 = x.index(locator3)
            xcoord.append(float(x[loc1:loc2-1]))
            ycoord.append(float(x[loc2:loc3]))
            radius.append(float(x[loc3+1:loc4]))
        except:
            print('Line - {}'.format(x))
    return [xcoord,ycoord,radius]

In [7]:
def plotMolesOnly(image,moleLocFile):
    imdata = getImageData(image,0)
    moleLocs = ds9File2Lists(moleLocFile)
    alignlocs = moleLocs
    mask = []
    #maskData = []
    for mole in range(len(alignlocs[0])):
        centre = PixCoord(alignlocs[0][mole],alignlocs[1][mole])
        reg = CirclePixelRegion(centre,alignlocs[2][mole])
        mask.append(reg.to_mask())
        #maskData.append(mask[mole].cutout(imdata)*mask[mole].data)
    
    fig, [ax1,ax2] = plt.subplots(1,2,figsize=(10,20))
    ax1.imshow(imdata)
    for x in range(len(mask)):
        numbering = mpl_text.Text(x=alignlocs[0][x],y=alignlocs[1][x],text=str(x),color='r')
        ax1.add_artist(mask[x].bbox.as_artist(facecolor='none', edgecolor='white'))
        ax1.add_artist(numbering)
    #ax2.imshow(zeros)
    ax2.imshow(imdata)
    plt.show()

In [172]:
def cosineRule(masterStar,star1,star2):
    a = ((masterStar[0]-star2[0])**2 + (masterStar[1]-star2[1])**2)**0.5
    b = ((star1[0]-masterStar[0])**2 + (star1[1]-masterStar[1])**2)**0.5
    c = (star1[0]-star2[0])**2 + (star1[1]-star2[1])**2
    C = np.arccos(((a**2) + (b**2) - c)/(2*a*b))
    #CRounded = round(C,1)
    return CRounded

In [161]:
a = 1.112 
print(round(a,2))

1.11


In [9]:
def findAllAnglesPerStar(table,starNo):
    masterStar = [table['X_IMAGE'][starNo],table['Y_IMAGE'][starNo]]
    allAngles = []
    for x in range(len(table)):
        if x != starNo:
            anglePerStar = []
            star1 = [table['X_IMAGE'][x],table['Y_IMAGE'][x]]
            for y in range(len(table)):
                if y != x and y!= starNo:
                    star2 = [table['X_IMAGE'][y],table['Y_IMAGE'][y]]
                    anglePerStar.append(cosineRule(masterStar,star1,star2))
            allAngles.append(anglePerStar)
    return allAngles

In [150]:
def distance(sources,object1,object2):
    """
    Find the distance between two objects in an image.

    :param sources: Table of object data
    :param object1: Index of first object in sources
    :param object2: Index of second object in sources
    :param imageNo: Image Number
    :return: Diagonal distance between the two objects
    """
    D = (((abs(sources['X_IMAGE'][object1]-sources['X_IMAGE'][object2])**2) + (abs(sources['Y_IMAGE'][object1]-sources['Y_IMAGE'][object2])**2))**0.5)
    Drounded = round(D,-1)
    #print([round(x/5,-1)*5 for x in a])
    return Drounded


In [11]:
def referenceDistances(sources,samplestars,imageNo):
    """
    Calculate the diagonal distance between every object in samplestars.

    :param sources: Table of object data
    :param samplestars: List containing the indexs of stars
    :param imageNo: reference image number
    :return: List containing a list for each object, comprised of the diagonal distance between that object and every object in samplestars
    """
    #referencedistances = [[]]*len(samplestars)
    referencedistances = [[] for x in samplestars]
    for y in range(len(samplestars)):
        refd = []
        for x in range(len(samplestars)):
            refd.append(distance(sources,samplestars[y],samplestars[x],imageNo))
        referencedistances[y] = refd #.append(distance(sources,samplestars[y],samplestars[x],imageNo))
    
    #astroprov.provcall([sources,samplestars,imageNo],[referencedistances],"referenceDistances_PythonCode2PythonCode_SQ_tmpl.provn","referenceDistances")
    return referencedistances


In [12]:
def findFullDistancesPerObject(sources,objectNo):
    """
    Find the diagonal distances between every object in the image.

    :param sources: Table of object data
    :param imageno: Image number
    :return: Digonal distance between every object in the image
    """

    fulldistances = []
    for y in range(len(sources)):
        fulldistances.append(distance(sources,objectNo,y)) 
    
    #astroprov.provcall([sources,imageno],[fulldistances],"findFullDistances_PythonCode2PythonCode_SQ_tmpl.provn","findFullDistances")
    #fulldistances = [round(number/10,1) * 10 for number in fulldistances] # rounding can be changed by changing the mul/div
    return fulldistances

In [13]:
def matchAngles(allAnglesPerObject,allAngles,matchFraction=0.6):
    percentageForMatch = 0.9

    # keep for now, inner join on an sql table might be better... ask Age about this?

    translationDictionary = {}

    for i,obj in enumerate(allAnglesPerObject): # loop over each object's angle set
        oneObjectDict = {}
        oneObjectLengths = {}
        for j,angleSet in enumerate(obj): # loop over one angle set
            matchLengths = []
            for k,ogObj in enumerate(allAngles): # loop over master angle set
                #result = all(elem in angleSet for elem in ogObj) # here is the problem-need to relax these constraints and change to any 
                result = any(elem in angleSet for elem in ogObj) # here is the problem-need to relax these constraints and change to any 
                result = [x for x in angleSet if x in ogObj]
                #originalLength = len(list(set(ogObj)))
                newLength = len(list(set(result)))
                #if newLength > matchFraction*originalLength: # this step may be unecessary
                if j not in oneObjectDict.keys():    
                    oneObjectDict[j] = k
                    oneObjectLengths[j] = newLength
                else:
                    if newLength > oneObjectLengths[j]:
                        oneObjectDict[j] = k
                        
        translationDictionary[i] = oneObjectDict
    return translationDictionary # values are -1 of their true values - this is accounted for later

In [104]:
def matchAngles(allAnglesPerObject,allAngles):
    percentageForMatch = 0.9

    # keep for now, inner join on an sql table might be better... ask Age about this?

    translationDictionary = {}

    for i,obj in enumerate(allAnglesPerObject): # loop over each object's angle set
        oneObjectDict = {}
        for j,angleSet in enumerate(obj): # loop over one angle set
            matchLengths = []
            for k,ogObj in enumerate(allAngles): # loop over master angle set
                result = all(elem in angleSet for elem in ogObj) # here is the problem-need to relax these constraints and change to any 
                #result = any(elem in angleSet for elem in ogObj) # here is the problem-need to relax these constraints and change to any 
                
                if result:
                    if j not in oneObjectDict.keys():
                        oneObjectDict[j] = [k]
                    elif k not in oneObjectDict[j]:
                        oneObjectDict[j].append(k)
        translationDictionary[i] = oneObjectDict
    return translationDictionary # values are -1 of their true values - this is accounted for later

In [85]:
test = {1:[1,2,3,4]}
test[1].append(5)
print(test)

{1: [1, 2, 3, 4, 5]}


In [19]:
# def confirmAngleMatchWithDistances(translationDictionary,refTab,referenceDistances,indexing):
#     trueMatchDict = {}
#     potentialMatches = []
#     potentialRef = []
#     for key,value in translationDictionary.items():
#         oneMatch = []
#         oneRef = []
#         if len(value) !=0:
#             for imgNo,sampleNo in value.items():
#                 objectDistance = distance(refTab,key,imgNo+1)
#                 if objectDistance == referenceDistances[indexing.index(sampleNo)]:
#                     oneMatch.append(imgNo+1)
#                     oneRef.append(sampleNo)
#         potentialMatches.append(oneMatch)
#         potentialRef.append(oneRef)

#     listLengths = [len(x) for x in potentialMatches]
#     bestMatch = max(listLengths)
#     bestMatch = [i for i,j in enumerate(listLengths) if j==bestMatch]
#     if len(bestMatch) > 1:
#         print('Currently {} potentail candidates, choosing the first'.format(len(bestMatch)))
#     trueMatchDict[0] = bestMatch[0]
#     for x in range(len(potentialMatches[bestMatch[0]])):
#         trueMatchDict[potentialRef[bestMatch[0]][x]] = potentialMatches[bestMatch[0]][x]
#     return trueMatchDict

In [245]:
def confirmAngleMatchWithDistances(translationDictionary,refTab,referenceDistances,indexing,closeness=5):
    trueMatchDict = {}
    potentialMatches = []
    potentialRef = []
    for key,value in translationDictionary.items():
        oneMatch = []
        oneRef = []
        if len(value) !=0:
            for imgNo,sampleNo in value.items():
                oneMatchSlim = [] # the slim versions of these lists remove a bug if two objects happen to have the 
                oneRefSlim = [] # same angle and distance to an object
                #print(sampleNo,imgNo)
                for multipleMatch in sampleNo:
                    objectDistance = distance(refTab,key,imgNo)
                    refDist = referenceDistances[indexing.index(multipleMatch)]
                    if objectDistance > (refDist-closeness) and objectDistance < (refDist+closeness):
                        oneMatchSlim.append(imgNo)
                        oneRefSlim.append(multipleMatch)
                if len(oneMatchSlim) > 0:
                    oneMatch.append(oneMatchSlim[0])
                    oneRef.append(oneRefSlim[0])
                    if len(list(set(oneMatchSlim))) > 1:
                        
                        print('Warning: Objects {} all match for sample object {}, selecting object {}'.format(oneMatchSlim,imgNo,oneMatchSlim[0]))
                        
        potentialMatches.append(oneMatch)
        potentialRef.append(oneRef)

    listLengths = [len(x) for x in potentialMatches]
    bestMatch = max(listLengths)
    print(listLengths)
    bestMatch = [i for i,j in enumerate(listLengths) if j==bestMatch]
    matchDictList = []
    
    print(bestMatch)
    for y in range(len(bestMatch)):
        trueMatchDict[0] = bestMatch[0]
        for x in range(len(potentialMatches[bestMatch[y]])):
            trueMatchDict[potentialRef[bestMatch[y]][x]] = potentialMatches[bestMatch[y]][x]
        matchDictList.append(trueMatchDict)
    if len(bestMatch) > 1:
        print('Currently {} potentail candidates, choosing the first'.format(len(bestMatch)))
#     trueMatchDict[0] = bestMatch[0]
#     for x in range(len(potentialMatches[bestMatch[0]])):
#         trueMatchDict[potentialRef[bestMatch[0]][x]] = potentialMatches[bestMatch[0]][x]
    return [matchDictList,potentialMatches,potentialRef]

In [16]:
#testSources = findTestSources([refTab],0,10000,0,10000,0)

In [17]:
# targetMoles = [44,37,35,34,40,41,20,16,15,14,10]
# targetMolesReal = [x+1 for x in targetMoles] 
# fullListofObjects = range(len(refTab))
# toBeRemoved = [x for x in fullListofObjects if x not in targetMoles]
# #print(len(fullListofObjects),len(toBeRemoved),len(testSources))
# refTab.remove_rows(toBeRemoved)
# sampleStars = refTab
# refTab = Table.read('/home/mj1e16/moleGazer/Photos/F/fitsImages/simMolePos/29018InvertedAverageCropped166.44444444444_21.8421052632_gauss_5.0_9x9.conv.cat',format='ascii.sextractor')

In [203]:
sampleTab = ascii.read('testSampleTable.tab',format='csv')
refTabTemp = Table.read('/home/mj1e16/moleGazer/Photos/F/fitsImages/simMolePos/29018InvertedAverageCropped166.44444444444_21.8421052632_gauss_5.0_9x9.conv.cat',format='ascii.sextractor')
leftIn = [0,8, 11, 15, 16, 17, 19, 21, 28, 38, 41, 42, 45, 48, 53]
allindex = refTab['NUMBER'].tolist()
remove = [x for x in allindex if x not in leftIn]
refTabTemp.remove_rows(remove)
sampleTab = refTabTemp
refTab = Table.read('/home/mj1e16/moleGazer/Photos/F/fitsImages/simMolePos/29018InvertedAverageCropped166.44444444444_21.8421052632_gauss_5.0_9x9.conv.cat',format='ascii.sextractor')


In [264]:
sampleTab = refTab[30:]

In [149]:
translationDict

{0: {7: [0, 1],
  8: [2, 3, 6, 8, 11, 12],
  9: [11, 12],
  10: [1],
  11: [1],
  12: [11],
  13: [5, 9, 10, 11, 12],
  14: [2, 3, 8, 11],
  15: [2, 3, 8, 11],
  16: [1, 2, 3, 4, 5, 8, 9, 10, 11, 12],
  18: [5, 9, 10, 11, 12],
  19: [11],
  20: [2, 3, 6, 8, 11, 12],
  21: [2, 3, 8, 11],
  22: [11],
  23: [5, 9, 10, 11, 12],
  24: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
  25: [2, 3, 6, 8, 11, 12],
  26: [11, 12],
  27: [5, 7, 9, 10, 11, 12],
  28: [11],
  29: [11],
  30: [2, 3, 6, 8, 11, 12],
  31: [11, 12],
  32: [5, 7, 9, 10, 11, 12],
  33: [5, 9, 10, 11, 12],
  34: [5, 9, 10, 11, 12],
  35: [11],
  36: [2, 3, 8, 11, 12],
  37: [2, 3, 8, 11],
  38: [5, 9, 10, 11, 12],
  39: [1, 2, 3, 4, 5, 8, 9, 10, 11, 12],
  40: [5, 9, 10, 11, 12],
  41: [5, 9, 10, 11, 12],
  43: [11, 12],
  44: [11],
  45: [2, 3, 8, 11],
  46: [2, 3, 8, 11],
  47: [11, 12],
  48: [11],
  49: [11],
  50: [5, 9, 10, 11, 12],
  51: [2, 3, 6, 8, 11, 12]},
 1: {0: [1, 2, 3, 4, 5, 8, 9, 10, 11, 12],
  2: [0],
  3: [1

In [265]:
allAngles = findAllAnglesPerStar(sampleTab,0) # all angles between master and sample stars
#allAngles = findAllAnglesPerStar(sampleStars,0) # all angles between master and sample stars

allAnglesPerObject = [] # nested list - each list represents one star chosen as the master star and the values within it are the angles between each other star
for x in range(len(refTab)):
    allAnglesPerObject.append(findAllAnglesPerStar(refTab,x))


In [266]:
translationDict = matchAngles(allAnglesPerObject,allAngles)

In [267]:
# referenceDistances = []
# indexing = []
# for x in range(1,len(sampleStars)):
#     referenceDistances.append(distance(sampleStars,0,x))
#     indexing.append(x-1)

referenceDistances = []
indexing = []
for x in range(len(sampleTab)):
    referenceDistances.append(distance(sampleTab,0,x))
    indexing.append(x)

In [268]:
leftIn = [8, 11, 15, 16, 17, 19, 21, 28, 38, 41, 42, 45, 48, 53]


In [269]:
len(sampleTab)

23

In [270]:
newtrueMatchDict = confirmAngleMatchWithDistances(translationDict,refTab,referenceDistances,indexing,closeness=50)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 2, 0, 2, 0, 1, 2, 3, 2, 5, 1, 1, 5, 3, 22, 2, 2, 5, 2, 0, 2, 4, 3, 1, 2, 2, 1, 5, 1, 0, 1, 1, 3, 1, 1, 4, 3]
[30]


In [271]:
for x in (newtrueMatchDict[1]):
    if len(x) ==14:
        print(x)

In [272]:
newtrueMatchDict[0]

[{0: 30,
  1: 31,
  2: 45,
  3: 33,
  4: 34,
  5: 35,
  6: 36,
  7: 37,
  8: 38,
  9: 39,
  10: 40,
  11: 41,
  12: 42,
  13: 43,
  14: 44,
  16: 46,
  17: 47,
  18: 48,
  19: 49,
  20: 50,
  21: 51}]

In [226]:
newtrueMatchDict[1][16]

[]

In [219]:
newtrueMatchDict[2][16]

[0, 2, 1, 5, 3, 6, 2, 6, 7, 5, 5, 7, 6, 6, 7]

In [88]:
translationDict

{0: {0: [0],
  1: [1],
  2: [2],
  3: [3],
  4: [4],
  5: [5],
  6: [6],
  7: [7],
  8: [8],
  24: [8],
  25: [8],
  27: [8],
  32: [8],
  51: [8]},
 1: {},
 2: {},
 3: {3: [6]},
 4: {},
 5: {},
 6: {},
 7: {},
 8: {},
 9: {},
 10: {},
 11: {6: [6]},
 12: {7: [1],
  9: [2, 4],
  13: [6],
  14: [6],
  15: [2],
  25: [2, 4],
  28: [4],
  29: [0, 6],
  31: [4],
  35: [0, 2, 4],
  39: [7],
  40: [7],
  42: [4],
  43: [4],
  47: [6],
  48: [2],
  51: [7]},
 13: {0: [4, 7, 8],
  1: [6],
  4: [8],
  6: [8],
  9: [2],
  10: [0, 6],
  15: [2],
  18: [2, 4],
  21: [6],
  24: [2, 4],
  26: [1, 2],
  28: [2],
  30: [1],
  40: [2, 4],
  47: [2],
  50: [1, 2],
  51: [4]},
 14: {9: [8], 12: [0], 17: [0]},
 15: {},
 16: {13: [0]},
 17: {7: [0, 8],
  17: [1, 6],
  19: [2, 4, 6],
  20: [0, 6],
  24: [0, 6],
  25: [6],
  28: [2, 4],
  29: [2, 4, 6],
  42: [6],
  44: [6],
  47: [6]},
 18: {15: [4],
  17: [1],
  18: [4],
  20: [2, 4],
  21: [6],
  25: [6],
  28: [2],
  35: [6],
  47: [2],
  48: [6]},
 19: 

In [137]:
refTab[15]

NUMBER,MAG_BEST,FLUX_MAX,X_IMAGE,Y_IMAGE,ELONGATION,ELLIPTICITY
,mag,ct,pix,pix,,
int64,float64,float64,float64,float64,float64,float64
16,-10.4845,51.63461,825.0093,1643.6385,1.229,0.186


In [138]:
sampleTab[0]

NUMBER,MAG_BEST,FLUX_MAX,X_IMAGE,Y_IMAGE,ELONGATION,ELLIPTICITY
,mag,ct,pix,pix,,
int64,float64,float64,float64,float64,float64,float64
17,-9.0449,45.6448,806.1796,1762.1414,1.445,0.308


In [46]:
refTab[52]

NUMBER,MAG_BEST,FLUX_MAX,X_IMAGE,Y_IMAGE,ELONGATION,ELLIPTICITY
,mag,ct,pix,pix,,
int64,float64,float64,float64,float64,float64,float64
53,-10.2325,39.3606,1095.95,3303.5989,1.961,0.49


In [47]:
sampleTab[-1]

NUMBER,MAG_BEST,FLUX_MAX,X_IMAGE,Y_IMAGE,ELONGATION,ELLIPTICITY,index,xLower,xUpper,xcoord,yLower,yUpper,ycoord
int64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64
53,-10.2325,39.3606,1095.95,3303.5989,1.961,0.49,11,1084.9415,1104.9415,1094.9415,3286.9145,3306.9145,3296.9145
